# SimParam

> The SimParam class is the cornerstone of a simulation, acting as a global container for parameters that are not specific to any single Population but govern the rules of the entire simulation. Just as the Population class holds the state of individuals, SimParam holds the "genetic laws" of the simulated world, such as trait architecture, genetic maps, and SNP chip definitions.



In [ ]:
#| default_exp sp

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass
from dataclasses import field
from typing import List, Optional, Dict, Callable

# NOTE: The following classes would typically be in their own files
# (e.g., `loci.py`, `traits.py`) but are included here for clarity.

@flax_dataclass(frozen=True)
class LociMap:
    """
    Defines a set of loci on the genome.
    
    Serves as a base class for SNP chips and Traits.
    """
    loci_per_chr: jnp.ndarray  # Shape: (nChr,)
    loci_loc: jnp.ndarray      # Shape: (total_loci_in_map,)
    name: str

    @property
    def n_loci(self) -> int:
        return self.loci_loc.shape[0]

@flax_dataclass(frozen=True)
class TraitA(LociMap):
    """
    Defines an additive trait.
    """
    add_eff: jnp.ndarray       # Shape: (n_loci,)
    intercept: float = 0.0

@flax_dataclass(frozen=True)
class SimParam:
    """
    A container for all global simulation parameters.
    ...
    """
    # --- Core Genome Structure ---
    gen_map: jnp.ndarray
    centromere: jnp.ndarray
    ploidy: int

    # --- Genetic Architecture ---
    traits: List[TraitA] = field(default_factory=list)
    snp_chips: List[LociMap] = field(default_factory=list)
    
    # --- Population State Reference ---
    # This is the crucial addition. It's optional to break the circular dependency.
    founderPop: Optional['Population'] = None

    # --- Simulation Control ---
    sexes: str = "no"          # "no", "yes_sys", "yes_rand"
    recomb_params: tuple = (2.6, 0.0, 0.0) # v, p, quadProb
    n_threads: int = 1

    # --- Pedigree & History Tracking ---
    track_pedigree: bool = False
    track_recomb: bool = False
    last_id: int = 0
    pedigree: Optional[jnp.ndarray] = None # Shape: (lastId, 3) for id, mother, father

    # --- Default Phenotyping Parameters ---
    var_e: Optional[jnp.ndarray] = None # Error variance

    @property
    def n_chr(self) -> int:
        return self.gen_map.shape[0]

    @property
    def n_loci_per_chr(self) -> jnp.ndarray:
        # Assuming genMap may be padded, we need a reliable way to get true loci counts
        # This could be stored separately or inferred if we have a non-padded source.
        # For now, let's assume the shape is accurate.
        return jnp.full((self.n_chr,), self.gen_map.shape[1])

    @property
    def n_traits(self) -> int:
        return len(self.traits)

    def __repr__(self) -> str:
        return (f"SimParam(nChr={self.n_chr}, nTraits={self.n_traits}, "
                f"ploidy={self.ploidy}, sexes='{self.sexes}')")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()